In [1]:
import pandas as pd
import numpy as np
import random

random.seed(33)

data = pd.read_csv("../../data/processed/phemernr2_dataset_with_tvt.csv")
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training


In [2]:
length = data.shape[0]
length

6425

In [3]:
data['label'].value_counts()

non-rumours    4023
rumours        2402
Name: label, dtype: int64

In [4]:
nr_topics = {}
nr_topics_us = {}
for i, df in data.iterrows():
    if df['label'] == 'non-rumours':
        if df['topic'] not in nr_topics:
            nr_topics[df['topic']] = 1
            nr_topics_us[df['topic']] = 0
        nr_topics[df['topic']] += 1
print(len(nr_topics))
print(nr_topics)

8
{'charliehebdo-all-rnr-threads': 1622, 'putinmissing-all-rnr-threads': 113, 'ferguson-all-rnr-threads': 860, 'ottawashooting-all-rnr-threads': 421, 'sydneysiege-all-rnr-threads': 700, 'germanwings-crash-all-rnr-threads': 232, 'gurlitt-all-rnr-threads': 78, 'prince-toronto-all-rnr-threads': 5}


In [5]:
data['topic'].unique().tolist()

['charliehebdo-all-rnr-threads',
 'germanwings-crash-all-rnr-threads',
 'putinmissing-all-rnr-threads',
 'ferguson-all-rnr-threads',
 'ottawashooting-all-rnr-threads',
 'sydneysiege-all-rnr-threads',
 'gurlitt-all-rnr-threads',
 'prince-toronto-all-rnr-threads',
 'ebola-essien-all-rnr-threads']

In [6]:
undersampling = []
weight = [0, 0, 1]
for i, df in data.iterrows():
    if df['label'] == "non-rumours":
        gacha = random.sample(weight, 1)[0]
        if gacha == 1:
            undersampling.append(True)
        else:
            undersampling.append(False)
    else:
        undersampling.append(True)
data['undersample'] = pd.Series(undersampling)
data['undersample'].value_counts()

True     3705
False    2720
Name: undersample, dtype: int64

In [7]:
data = data[data['undersample'] == True]
data.shape

(3705, 9)

In [11]:
nr_topics2 = {}
for i, df in data.iterrows():
    if df['label'] == 'non-rumours':
        if df['topic'] not in nr_topics2:
            nr_topics2[df['topic']] = 1
        nr_topics2[df['topic']] += 1
print(len(nr_topics2))
print(nr_topics2)

8
{'charliehebdo-all-rnr-threads': 513, 'ferguson-all-rnr-threads': 279, 'sydneysiege-all-rnr-threads': 230, 'germanwings-crash-all-rnr-threads': 86, 'ottawashooting-all-rnr-threads': 136, 'putinmissing-all-rnr-threads': 36, 'gurlitt-all-rnr-threads': 29, 'prince-toronto-all-rnr-threads': 2}


In [26]:
for k, v in nr_topics.items():
    print(f"{k.split('-')[0]}\t: {v}")

charliehebdo	: 1622
putinmissing	: 113
ferguson	: 860
ottawashooting	: 421
sydneysiege	: 700
germanwings	: 232
gurlitt	: 78
prince	: 5


In [31]:
for k, v in nr_topics.items():
    print(f"{k.split('-')[0]}\t: {v} | {nr_topics2[k]}")

charliehebdo	: 1622 | 513
putinmissing	: 113 | 36
ferguson	: 860 | 279
ottawashooting	: 421 | 136
sydneysiege	: 700 | 230
germanwings	: 232 | 86
gurlitt	: 78 | 29
prince	: 5 | 2


In [20]:
total = sum([v for k, v in nr_topics.items()])
total2 = sum([v for k, v in nr_topics2.items()])
for k, v in nr_topics.items():
    print(f"{k.split('-')[0]}\t: original {round(v/total, 3)} | {round(nr_topics2[k]/total2, 3)} undersampled")

charliehebdo	: original 0.402/0.391 undersampled
putinmissing	: original 0.028/0.027 undersampled
ferguson	: original 0.213/0.213 undersampled
ottawashooting	: original 0.104/0.104 undersampled
sydneysiege	: original 0.174/0.175 undersampled
germanwings	: original 0.058/0.066 undersampled
gurlitt	: original 0.019/0.022 undersampled
prince	: original 0.001/0.002 undersampled


In [33]:
data = data[['tweet_id', 'tweet_text', 'label', 'label2', 'topic']]
print(data.shape)
data.head()

(3705, 5)


,tweet_id,tweet_text,label,label2,topic
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads
7,552822434639126528,"""You tell the media it was al-Qaeda in Yemen"" ...",rumours,unverified,charliehebdo-all-rnr-threads


In [34]:
data['label'].value_counts()

rumours        2402
non-rumours    1303
Name: label, dtype: int64

In [35]:
data.to_csv("../../data/processed/phemernr3_dataset_with_tvt.csv", index=False)